# Deep Learning approach to the Titanic problem

In [ ]:
%env PYTHONHASHSEED=0

import os
DEVMODE = os.getenv("KAGGLE_MODE") == "DEV"
print(f"DEV MODE: {DEVMODE}")

# Define seed to reprodicibility of random generation
SEED = 42
DEV_SPLIT = 0.2

In [ ]:
import importlib
if importlib.util.find_spec("comet_ml"):
    from comet_ml import Experiment
    experiment = Experiment(project_name="titanic")
else:
    experiment = None

In [ ]:
import numpy as np
import pandas as pd

# To display all the columns from left to right without breaking into next line.
pd.set_option("display.width", 1500)
pd.plotting.register_matplotlib_converters()

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

In [ ]:
import random as python_random

# Make sure Keras produces reproducible results.

np.random.seed(SEED)
python_random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print(physical_devices)
for device in (physical_devices or []):
    tf.config.experimental.set_memory_growth(device, True)

## Load data and split into train/dev sets

In [ ]:
from titanic.titanic_data import load_titanic_data, split_data, get_data_preprocessor

X_train_full, y_train_full, X_pred = load_titanic_data()
X_train, X_valid, y_train, y_valid = split_data(X_train_full, y_train_full, test_size=DEV_SPLIT, random_state=SEED)

## Define pre-processing of the data

In [ ]:
preprocessor, preprocessed_column_names = get_data_preprocessor()

In [ ]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), index=X_train.index, columns=preprocessed_column_names)
X_valid = pd.DataFrame(preprocessor.transform(X_valid), index=X_valid.index, columns=preprocessed_column_names)
X_pred = pd.DataFrame(preprocessor.transform(X_pred), index=X_pred.index, columns=preprocessed_column_names)

In [ ]:
X_train.head()

## DL model using Keras

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name="tp"),
      keras.metrics.FalsePositives(name="fp"),
      keras.metrics.TrueNegatives(name="tn"),
      keras.metrics.FalseNegatives(name="fn"),
      keras.metrics.BinaryAccuracy(name="accuracy"),
      keras.metrics.Precision(name="precision"),
      keras.metrics.Recall(name="recall"),
      keras.metrics.AUC(name="auc"),
]

def get_model(input_size):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Input
    from tensorflow.keras.regularizers import l2

    model = Sequential([
        Input(shape=(input_size,)),
        Dense(40, activation="relu", kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001), kernel_initializer="he_uniform"),
        Dropout(0.2),
        Dense(56, activation="tanh", kernel_regularizer=l2(0.1), bias_regularizer=l2(0.1), kernel_initializer="glorot_uniform"),
        Dropout(0.3),
        Dense(56, activation="relu", kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_initializer="he_uniform"),
        Dropout(0.3),
        Dense(16, activation="relu", kernel_regularizer=l2(0.001), bias_regularizer=l2(0.001), kernel_initializer="he_uniform"),
        Dropout(0.45),
        Dense(16, activation="tanh", kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_initializer="glorot_uniform"),
        Dropout(0.35),
        Dense(8, activation="relu", kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), kernel_initializer="he_uniform"),
        Dropout(0.4),
        Dense(1, activation="sigmoid")
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(1e-3), metrics=METRICS, loss="binary_crossentropy")
    
    return model

In [ ]:
from titanic.titanic_data import get_class_weights

# Create a new model each time before running training (otherwise new trainings would just be on already trained model)
model = get_model(X_train.shape[1])

history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_data=(X_valid, y_valid), class_weight=get_class_weights(y_train), verbose=1)

## Results of the DL model

In [ ]:
from utils.visualising import plot_model_history

plot_model_history(history)

In [ ]:
from utils.visualising import draw_confusion_matrix

print("Train evaluation:")
evaluation_train = model.evaluate(X_train, y_train, verbose=2)
draw_confusion_matrix(evaluation_train, "train")

In [ ]:
print("Valid evaluation:")
evaluation_valid = model.evaluate(X_valid, y_valid, verbose=2)
draw_confusion_matrix(evaluation_valid, "valid")

## Predict with DL model

In [ ]:
from utils.predicting import store_predictions

In [ ]:
store_predictions(model, X_pred=X_pred, index=X_pred.index, submission_name="dl")

In [ ]:
if experiment:
    experiment.end()